# IC 4.6. Combining & Exporting Data Submission

### 0. List of Contents
#### 1. Importing Libraries and Dataframes
#### 2. Preparation for Combining Data
#### 3. Combining Data
#### 4. Cleaning up and exporting

### 1. Importing Libraries and Dataframes

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Creating path
path = r'C:\Users\User\Documents\Tanulok\CareerFoundry\2. Data Immersion\4. Python\Instacart Basket Analysis\02 Data'

In [3]:
# Importing df_orders_products_combined
df_orders_products_combined = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_combined.pkl'))

In [4]:
# Importing df_prods
df_prods = pd.read_csv(os.path.join(path, 'Prepared Data', 'products_checked.csv'),index_col=False)

### 2. Preparation for Combining Data

In [5]:
# Checking outcome
df_prods.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


In [6]:
# Checking outcome
df_orders_products_combined.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,prior,1,2,8,NaN,196,1,0,both
1,2539329,1,prior,1,2,8,NaN,14084,2,0,both
2,2539329,1,prior,1,2,8,NaN,12427,3,0,both
3,2539329,1,prior,1,2,8,NaN,26088,4,0,both
4,2539329,1,prior,1,2,8,NaN,26405,5,0,both


In [7]:
# checking the shape
df_prods.shape

(49672, 6)

In [8]:
# checking the shape
df_orders_products_combined.shape

(32434489, 11)

### 3. Combining Data

##### the two dfs have different length of columns and rows, but have a common identifier -> product_id -> hence merging them is the best solution. Besides the order data set is bigger, which makes sense as the number of products Instacart has is limited, but the number of orders could be unlimited, hence I'll do left join.

In [9]:
#removing _merge column in preparation for merge
df_orders_products_combined=df_orders_products_combined.drop(columns=['_merge'])

In [10]:
#merging
df_orders_products_merged = pd.merge(df_orders_products_combined, df_prods, on = 'product_id', how = 'left', indicator = True)

In [11]:
#Confirm the results of the merge using the merge flag
df_orders_products_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,prior,1,2,8,NaN,196,1,0,195.0,Soda,77.0,7.0,9.0,both
1,2539329,1,prior,1,2,8,NaN,14084,2,0,14084.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both
2,2539329,1,prior,1,2,8,NaN,12427,3,0,12427.0,Original Beef Jerky,23.0,19.0,4.4,both
3,2539329,1,prior,1,2,8,NaN,26088,4,0,26089.0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both
4,2539329,1,prior,1,2,8,NaN,26405,5,0,26406.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both


In [13]:
#Confirm the results of the merge using the merge flag with frequency
df_orders_products_merged['_merge'].value_counts()

both          32404859
left_only        30200
right_only           0
Name: _merge, dtype: int64

### 4. Cleaning up and exporting

In [15]:
#eval_set should have been removed earlier and Unnamed: 0 is an old index column which is confusing -> they need to be removed from the clean dataset
df_orders_products_merged = df_orders_products_merged.drop(columns = ['eval_set' ,  'Unnamed: 0'])

In [17]:
#check outcome
df_orders_products_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77.0,7.0,9.0,both
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,both
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both


In [18]:
# Export data to pkl due to size

df_orders_products_merged.to_pickle(os.path.join(path,'Prepared Data', 'df_orders_products_merged.pkl'))